In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from models_mse import LstmMse
from models_mle import LstmMle_1, LstmMle_2, LstmMle_3
from data_preperator import DataPreperatorPrediction
from data_set import DataSet
from predictor import PredictorMse, PredictorMle
from predictor_cell_state import PredictorMleCellState
from anomaly_classifier import AnomalyClassifier

# Mode: 

In [2]:
mode = "MLE"

## Parameters phm data

In [17]:
param = {
    "data" : {
        "path" : '../../data/phm_data_challenge/recipe/dataset_for_each_recipe/test/test_recipe_67.csv',
        "droped_feature" : ["stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION", "ROTATIONSPEED"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/{}_model/phm67_id36649.pt".format(mode),
        "input_size" : 12,
        "n_hidden_lstm" : 13,
        "n_hidden_fc_1" : 55,
        "n_hidden_fc_2" : 25,
        "sequence_size" : 21,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
    "results": {
        "path" : "../visualisation/files/prediction/{}/phm_67.csv".format(mode),
    }
}

## Parameters artifical data

In [3]:
param = {
    "data" : {
        "path" : '../../data/artifical_signals/artifical_2_signals_errors.csv',
        "droped_feature" : ["anomaly"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/{}_model/art2_id82478.pt".format(mode),
        "input_size" : 2,
        "n_hidden_lstm" : 8,
        "n_hidden_fc_1" : 55,
        "n_hidden_fc_2" : 25,
        "sequence_size" : 21,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
    "results": {
        "path" : "../visualisation/files/prediction/{}/artfic.csv".format(mode),
    }
}

## Parameters cpps data

In [10]:
param = {
    "data" : {
        "path" : '../../data/cpps_degradation_new/data_obs10/test/obs_space_error_sinusiod_preprocessed.csv',
        "droped_feature" : ["status"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/{}_model/cpps_id61644.pt".format(mode),
        "input_size" : 10,
        "n_hidden_lstm" : 21,
        "n_hidden_fc_1" : 55,
        "n_hidden_fc_2" : 25,
        "sequence_size" : 8,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
    "results": {
        "path" : "../visualisation/files/prediction/{}/cpps.csv".format(mode), 
    }
}

## Standarize Data
First we have to apply normalisation to data. That is because the model works on the representation given by its input vectors. The scale of those numbers is part of the representation.
We should apply the exact same scaling as for training data. That means storing the scale and offset used with your training data, and using that again. <br>
__The mean and variance for each feature of the training data with which the model was trained (stake: 0.75):__

### Mean and variance from phm dataset - recipe 67 (29.219 samples)

In [18]:
mean_training_data = [0.00236702, 0.3953089,0.48772743 ,0.3857511, 0.49987399, 0.06291772, -0.03491417,
                      0.43371134,  0.23365129, -0.06136357, -0.12245359,  0.2516167]
var_training_data =[3.62155978e-04, 7.33502893e-01, 8.30760891e-01, 7.06805763e-01, 9.44752420e-01,
                    4.07861536e-01, 1.61079596e-01, 8.46321709e-01, 3.65172841e-01, 6.37131077e-01,
                    4.60727666e-11, 9.93519995e-01]

### Mean and variance from artifical dataset

In [4]:
mean_training_data= [-0.00526595, -0.00968424]
var_training_data = [49.30277603, 0.4232726 ]

### Mean and Variance fromm NEW cpps dataset

In [11]:
mean_training_data = [-0.06942446, -0.03770532, -0.01556928, -0.04071856, -0.04357261, -0.06645444, -0.06779275, 
                      -0.01516744, -0.04079089, -0.03428566]
var_training_data = [0.15114678, 0.1605483,  0.07761351, 0.06546521, 0.09442645, 0.10429262,
                     0.09060877, 0.07936068, 0.06720505, 0.09224595]

## Create DataLoader

In [19]:
data_preperator = DataPreperatorPrediction(path=param['data']['path'], 
                                           ignored_features = param["data"]["droped_feature"],
                                           mean_training_data=mean_training_data, 
                                           var_training_data=var_training_data, 
                                           first_order_difference=False 
                                          )                                  
preprocessed_data = data_preperator.prepare_data()
print(preprocessed_data.shape)

dataset = DataSet(preprocessed_data, 
                  timesteps=param["model"]["sequence_size"])
data_loader = DataLoader(dataset, 
                         batch_size=param['model']['batch_size'], 
                         num_workers=0, 
                         shuffle=False, 
                         drop_last=True)

(488363, 13)


In [20]:
for batch_idx, data in enumerate(data_loader):
    x,y = data
    print('Data of batch: {}'.format(batch_idx))
    print("Size of input data: {}".format(x.size()))
    print("Size of target data: {}".format(y.size()))
    if batch_idx >=1: break

Data of batch: 0
Size of input data: torch.Size([50, 21, 13])
Size of target data: torch.Size([50, 13])
Data of batch: 1
Size of input data: torch.Size([50, 21, 13])
Size of target data: torch.Size([50, 13])


## Define Model and load Parameters of trained model
### Model for MSE 

In [ ]:
model = LstmMse(batch_size=param['model']['batch_size'], 
                input_dim=param['model']['input_size'], 
                n_hidden_lstm=param['model']['n_hidden_lstm'], 
                n_hidden_fc=param['model']['n_hidden_fc_1'], 
                n_layers=param['model']['lstm_layer'], 
                dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                dropout_rate_fc= param['model']['dropout_rate_fc']
                )

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

### Model for MLE
#### Architecture 1 - One FCNN (2 layers, last splited for mu and sigma)

In [21]:
model = LstmMle_1(batch_size=param['model']['batch_size'], 
                 input_dim=param['model']['input_size'], 
                 n_hidden_lstm=param['model']['n_hidden_lstm'], 
                 n_hidden_fc_1=param['model']['n_hidden_fc_1'], 
                 n_layers=param['model']['lstm_layer'], 
                 dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                 dropout_rate_fc= param['model']['dropout_rate_fc'],
                 K = param['model']['K'])

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

#### Architecture 2 - Two seperate FCNN for mu and sigma each (2 layers)

In [ ]:
model = LstmMle_2(batch_size=param['model']['batch_size'], 
                 input_dim=param['model']['input_size'], 
                 n_hidden_lstm=param['model']['n_hidden_lstm'], 
                 n_layers=param['model']['lstm_layer'],
                 dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                 dropout_rate_fc= param['model']['dropout_rate_fc'],
                 n_hidden_fc_1=param['model']['n_hidden_fc_1'],
                 K = K_phase_1
                 )

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

#### Architecture 3 - Two seperate FCNN for mu and sigma each (3 layers)

In [ ]:
model = LstmMle_3(batch_size=param['model']['batch_size'], 
                 input_dim=param['model']['input_size'], 
                 n_hidden_lstm=param['model']['n_hidden_lstm'], 
                 n_hidden_fc_1=param['model']['n_hidden_fc_1'], 
                 n_hidden_fc_2=param['model']['n_hidden_fc_2'],
                 n_layers=param['model']['lstm_layer'], 
                 dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                 dropout_rate_fc= param['model']['dropout_rate_fc'],
                 K = param['model']['K'])

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

## Initialize Predictor
### For MSE Model

In [ ]:
predictor = PredictorMse(model=model,
                         path_data=param["data"]["path"],
                         columns_to_ignore=param["data"]["droped_feature"]
                         )

### For MLE Model

In [22]:
predictor = PredictorMle(model=model,
                         path_data=param["data"]["path"],
                         columns_to_ignore=param["data"]["droped_feature"],
                         )

## Predict

In [23]:
print("Start predicting.")
# Write header
with open(param["results"]["path"], "a+") as file:
            [file.write(column+";") for column in predictor.create_column_names_result()]
            file.write("\n")

for batch_number, (input_data, target_data) in enumerate(data_loader):
    # Predict sensor values in mini-batches
    batch_results = predictor.predict(input_data, target_data)
    
    # Write results to csv file
    with open(param["results"]["path"], "a") as file:
        for batch in batch_results:
            # Each result component of a singe prediction (ID, target, prediction, loss, latent space ...) is stored in lists
            # thus we have to unpack the list and seperate values with ;
            for value in batch:
                file.write(str(value)+";")
            file.write("\n")

    # Print status 
    if (batch_number*param['model']['batch_size'])%5000 == 0:
        print("Current status: " + str(param['model']['batch_size']*batch_number) + " samples are predicted.")

print("End of prediction.")

Start predicting.
Current status: 0 samples are predicted.
Current status: 5000 samples are predicted.
Current status: 10000 samples are predicted.
Current status: 15000 samples are predicted.
Current status: 20000 samples are predicted.
Current status: 25000 samples are predicted.
Current status: 30000 samples are predicted.
Current status: 35000 samples are predicted.
Current status: 40000 samples are predicted.
Current status: 45000 samples are predicted.
Current status: 50000 samples are predicted.
Current status: 55000 samples are predicted.
Current status: 60000 samples are predicted.
Current status: 65000 samples are predicted.
Current status: 70000 samples are predicted.
Current status: 75000 samples are predicted.
Current status: 80000 samples are predicted.
Current status: 85000 samples are predicted.
Current status: 90000 samples are predicted.
Current status: 95000 samples are predicted.
Current status: 100000 samples are predicted.
Current status: 105000 samples are predic